In [2]:
from flask import Flask, request, jsonify, render_template
import sqlite3
import re

app = Flask(__name__)

def create_database():
    conn = sqlite3.connect("company.db")
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Employees (
            ID INTEGER PRIMARY KEY,
            Name TEXT,
            Department TEXT,
            Salary INTEGER,
            Hire_Date TEXT
        )
    """)
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Departments (
            ID INTEGER PRIMARY KEY,
            Name TEXT,
            Manager TEXT
        )
    """)
    
    cursor.execute("DELETE FROM Employees")
    cursor.execute("DELETE FROM Departments")
    
    cursor.executemany("""
        INSERT INTO Employees (ID, Name, Department, Salary, Hire_Date)
        VALUES (?, ?, ?, ?, ?)
    """, [
        (1, 'Alice', 'Sales', 50000, '2021-01-15'),
        (2, 'Bob', 'Engineering', 70000, '2020-06-10'),
        (3, 'Charlie', 'Marketing', 60000, '2022-03-20')
    ])
    
    cursor.executemany("""
        INSERT INTO Departments (ID, Name, Manager)
        VALUES (?, ?, ?)
    """, [
        (1, 'Sales', 'Alice'),
        (2, 'Engineering', 'Bob'),
        (3, 'Marketing', 'Charlie')
    ])
    
    conn.commit()
    conn.close()

def get_response(query):
    conn = sqlite3.connect("company.db")
    cursor = conn.cursor()
    query = query.lower()
    
    try:
        if match := re.search(r"show me all employees in the (\w+) department", query):
            department = match.group(1).capitalize()
            cursor.execute("SELECT Name FROM Employees WHERE Department = ?", (department,))
            result = cursor.fetchall()
            return [row[0] for row in result] or ["No employees found"]
        
        elif match := re.search(r"who is the manager of the (\w+) department", query):
            department = match.group(1).capitalize()
            cursor.execute("SELECT Manager FROM Departments WHERE Name = ?", (department,))
            result = cursor.fetchone()
            return result[0] if result else "Department not found"
        
        elif match := re.search(r"list all employees hired after (\d{4}-\d{2}-\d{2})", query):
            date = match.group(1)
            cursor.execute("SELECT Name FROM Employees WHERE Hire_Date > ?", (date,))
            result = cursor.fetchall()
            return [row[0] for row in result] or ["No employees found"]
        
        elif match := re.search(r"what is the total salary expense for the (\w+) department", query):
            department = match.group(1).capitalize()
            cursor.execute("SELECT SUM(Salary) FROM Employees WHERE Department = ?", (department,))
            result = cursor.fetchone()
            return f"Total salary expense: {result[0]}" if result[0] else "Department not found or no salary data"
        
        else:
            return "Sorry, I didn't understand the query."
    
    except sqlite3.Error as e:
        return f"Database error: {e}"
    
    finally:
        conn.close()

if __name__ == "__main__":
    create_database()
    print("Chat Assistant Ready! Type your query:")
    while True:
        user_query = input("You: ")
        if user_query.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        response = get_response(user_query)
        print("Bot:", response)
if __name__ == "__main__":
    create_database()
    app.run(debug=True)



Chat Assistant Ready! Type your query:
You: Who is the manager of the Marketing department?
Bot: Charlie
You: exit
Goodbye!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
